# word2vec-nlp-tutorial

Leaning some NLP basics.

## Module Imports

In [27]:
import os
import re
from bs4 import BeautifulSoup             

import nltk
import pandas as pd
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm

/Users/chenjun/opt/miniconda3/envs/py38_tf29/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Configs

In [ ]:
DATA_DIR = '../data/word2vec-nlp-tutorial'

In [13]:
nltk.data.path.append('/Users/chenjun/workspace/nltk_data')

## EDA

In [3]:
os.listdir(DATA_DIR)

['.DS_Store',
 'sampleSubmission.csv',
 'labeledTrainData.tsv',
 'testData.tsv',
 'unlabeledTrainData.tsv']

In [18]:
labeled_train_data = pd.read_csv(os.path.join(DATA_DIR, 'labeledTrainData.tsv'), header=0, delimiter='\t', quoting=3)

In [19]:
labeled_train_data.shape

(25000, 3)

In [20]:
labeled_train_data.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


## Data Preprocessing

In [14]:
STOP_WORDS = set(stopwords.words('english'))

In [15]:
def clean_review(raw_review: str) -> List[str]:
    # Remove HTML markups
    cleaned_review = BeautifulSoup(raw_review).get_text()
    
    # Remove punctuations
    cleaned_review = re.sub('[^a-zA-Z]', ' ', cleaned_review)
    
    # Lower and split
    words = cleaned_review.lower().split()
    
    # Remove stop wrods
    non_stop_words = [word for word in words if word not in STOP_WORDS]
    
    return ' '.join(non_stop_words)

In [21]:
clean_review(labeled_train_data['review'][0])

'stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mj feeling towards press also obvious message drugs bad kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice actual feature film bit finally starts minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond mj overheard plans nah joe pesci character ranted wanted people know supplying drugs etc dunno maybe hates mj music lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually directors hate workin

In [23]:
training_data = []
for idx in tqdm(range(labeled_train_data.shape[0])):
    training_data.append(clean_review(labeled_train_data['review'][idx]))

  0%|                                                                                                      | 0/25000 [00:00<?, ?it/s]/Users/chenjun/opt/miniconda3/envs/py38_tf29/lib/python3.8/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
100%|████████████████████████████████████████████████████████████████████████████████████████| 25000/25000 [00:03<00:00, 7762.60it/s]


In [32]:
clean_train_reviews = training_data

In [24]:
len(training_data)

25000

In [25]:
training_data[0]

'stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mj feeling towards press also obvious message drugs bad kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice actual feature film bit finally starts minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond mj overheard plans nah joe pesci character ranted wanted people know supplying drugs etc dunno maybe hates mj music lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually directors hate workin

## Feature Engineering

In [33]:
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(clean_train_reviews)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()

## Model

In [35]:
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit(train_data_features, labeled_train_data["sentiment"] )

## Creating Submission

In [39]:
# Read the test data
test = pd.read_csv(os.path.join(DATA_DIR, "testData.tsv"), header=0, delimiter="\t", quoting=3 )

# Create an empty list and append the clean reviews one by one
num_reviews = len(test["review"])
clean_test_reviews = [] 

for i in tqdm(range(0,num_reviews)):
    cleaned_review = clean_review( test["review"][i] )
    clean_test_reviews.append( cleaned_review )

# Get a bag of words for the test set, and convert to a numpy array
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

# Use the random forest to make sentiment label predictions
result = forest.predict(test_data_features)

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

# Use pandas to write the comma-separated output file
output.to_csv( "Bag_of_Words_model.csv", index=False, quoting=3 )

  0%|                                                                                                                                                                               | 0/25000 [00:00<?, ?it/s]/Users/chenjun/opt/miniconda3/envs/py38_tf29/lib/python3.8/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25000/25000 [00:03<00:00, 8091.08it/s]
